In [1]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from numpy import arange,array,ones,linalg
from pylab import plot,show

In [2]:
ratings = pd.read_csv('Data/ratings.csv')
print(ratings.head(10))

   userId  movieId  rating   timestamp
0       1       31     2.5  1260759144
1       1     1029     3.0  1260759179
2       1     1061     3.0  1260759182
3       1     1129     2.0  1260759185
4       1     1172     4.0  1260759205
5       1     1263     2.0  1260759151
6       1     1287     2.0  1260759187
7       1     1293     2.0  1260759148
8       1     1339     3.5  1260759125
9       1     1343     2.0  1260759131


In [3]:
movies = pd.read_csv('Data/movies.csv')
print(movies)

      movieId                                              title  \
0           1                                   Toy Story (1995)   
1           2                                     Jumanji (1995)   
2           3                            Grumpier Old Men (1995)   
3           4                           Waiting to Exhale (1995)   
4           5                 Father of the Bride Part II (1995)   
5           6                                        Heat (1995)   
6           7                                     Sabrina (1995)   
7           8                                Tom and Huck (1995)   
8           9                                Sudden Death (1995)   
9          10                                   GoldenEye (1995)   
10         11                     American President, The (1995)   
11         12                 Dracula: Dead and Loving It (1995)   
12         13                                       Balto (1995)   
13         14                                   

In [4]:
movies=movies.set_index('movieId').to_dict()

In [5]:
movies=movies['title']
print(movies)

{1: 'Toy Story (1995)', 2: 'Jumanji (1995)', 3: 'Grumpier Old Men (1995)', 4: 'Waiting to Exhale (1995)', 5: 'Father of the Bride Part II (1995)', 6: 'Heat (1995)', 7: 'Sabrina (1995)', 8: 'Tom and Huck (1995)', 9: 'Sudden Death (1995)', 10: 'GoldenEye (1995)', 11: 'American President, The (1995)', 12: 'Dracula: Dead and Loving It (1995)', 13: 'Balto (1995)', 14: 'Nixon (1995)', 15: 'Cutthroat Island (1995)', 16: 'Casino (1995)', 17: 'Sense and Sensibility (1995)', 18: 'Four Rooms (1995)', 19: 'Ace Ventura: When Nature Calls (1995)', 20: 'Money Train (1995)', 21: 'Get Shorty (1995)', 22: 'Copycat (1995)', 23: 'Assassins (1995)', 24: 'Powder (1995)', 25: 'Leaving Las Vegas (1995)', 26: 'Othello (1995)', 27: 'Now and Then (1995)', 28: 'Persuasion (1995)', 29: 'City of Lost Children, The (Cit\xc3\xa9 des enfants perdus, La) (1995)', 30: 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)', 31: 'Dangerous Minds (1995)', 32: 'Twelve Monkeys (a.k.a. 12 Monkeys) (1995)', 34: 'Babe (1995)', 

In [6]:
ratings = ratings.iloc[9:,:3].values
userRatings = {}
for rating in ratings:
    userRatings.setdefault(rating[0],{})
    userRatings[rating[0]][rating[1]] = rating[2]
print(userRatings)

{1.0: {1953.0: 4.0, 2150.0: 3.0, 3671.0: 3.0, 2193.0: 2.0, 2294.0: 2.0, 2455.0: 2.5, 2968.0: 1.0, 2105.0: 4.0, 1371.0: 2.5, 1405.0: 1.0, 1343.0: 2.0}, 2.0: {515.0: 4.0, 261.0: 4.0, 265.0: 5.0, 10.0: 4.0, 527.0: 4.0, 144.0: 3.0, 17.0: 5.0, 405.0: 2.0, 150.0: 5.0, 537.0: 4.0, 485.0: 3.0, 153.0: 4.0, 410.0: 3.0, 367.0: 3.0, 497.0: 3.0, 161.0: 3.0, 539.0: 3.0, 292.0: 3.0, 165.0: 3.0, 550.0: 3.0, 39.0: 5.0, 168.0: 3.0, 551.0: 5.0, 300.0: 3.0, 500.0: 4.0, 47.0: 4.0, 50.0: 4.0, 52.0: 3.0, 350.0: 4.0, 585.0: 5.0, 185.0: 3.0, 186.0: 3.0, 509.0: 4.0, 266.0: 5.0, 62.0: 3.0, 319.0: 1.0, 272.0: 3.0, 454.0: 4.0, 552.0: 3.0, 457.0: 3.0, 586.0: 3.0, 587.0: 3.0, 588.0: 3.0, 589.0: 5.0, 590.0: 5.0, 208.0: 3.0, 593.0: 3.0, 339.0: 3.0, 468.0: 4.0, 592.0: 5.0, 474.0: 2.0, 314.0: 4.0, 222.0: 5.0, 223.0: 1.0, 480.0: 4.0, 225.0: 3.0, 356.0: 3.0, 357.0: 3.0, 273.0: 4.0, 616.0: 3.0, 235.0: 3.0, 364.0: 3.0, 720.0: 4.0, 110.0: 4.0, 317.0: 2.0, 296.0: 4.0, 370.0: 2.0, 371.0: 3.0, 372.0: 3.0, 349.0: 4.0, 248.0: 3.0

In [7]:
from math import sqrt
def pearsonSimilarity(person1,person2):
	common=[]
	for item in userRatings[person1]:
		if item in userRatings[person2]:
			common.append(item)
	
	if len(common)==0 :
		return 0
	
	sum1=0
	sum2=0
	prod12=0
	sumSquare1=0
	sumSquare2=0
	for item in common:
		sum1=sum1+userRatings[person1][item]
		sumSquare1=sumSquare1+(pow(userRatings[person1][item],2))	
		sumSquare2=sumSquare2+(pow(userRatings[person2][item],2))
		sum2=sum2+userRatings[person2][item]
		prod12=prod12+userRatings[person1][item]*userRatings[person2][item]
	
	num=(len(common)*prod12)-(sum1*sum2)
	deno=(len(common)*sumSquare1-pow(sum1,2))*(len(common)*sumSquare2-pow(sum2,2))
	
	if (deno==0):
		return 0
	return num/(sqrt(deno))	

In [8]:
def getRecommendations(critics,person):
	similaritySum={}
	totalSum={}
	for otherPerson in critics:
		if otherPerson!=person:
			similarityScore=pearsonSimilarity(person,otherPerson)
			if similarityScore<0:
				continue
			for movie in critics[otherPerson]:
				if movie not in critics[person]:
					totalSum.setdefault(movie,0)
					totalSum[movie]=totalSum[movie]+similarityScore*critics[otherPerson][movie]
					similaritySum.setdefault(movie,0)
					similaritySum[movie]=similaritySum[movie]+similarityScore
	
	#print(similaritySum)
	#print(totalSum)
	recommendedMovies=[]
	for movie in totalSum:
		if similaritySum[movie]==0:
			recommendedMovies.append((0,movie))
		else:
			recommendedMovies.append((totalSum[movie]/similaritySum[movie],movie))
	recommendedMovies.sort()
	recommendedMovies.reverse()
	return recommendedMovies
	

In [9]:
recommendedMovieIds=getRecommendations(userRatings,1)
print(recommendedMovieIds)

[(5.000000000000001, 3153.0), (5.0, 163949.0), (5.0, 150856.0), (5.0, 114464.0), (5.0, 112577.0), (5.0, 104119.0), (5.0, 101850.0), (5.0, 96075.0), (5.0, 95113.0), (5.0, 94466.0), (5.0, 86487.0), (5.0, 80553.0), (5.0, 77291.0), (5.0, 74089.0), (5.0, 71390.0), (5.0, 63836.0), (5.0, 60384.0), (5.0, 58649.0), (5.0, 55063.0), (5.0, 53887.0), (5.0, 42900.0), (5.0, 40226.0), (5.0, 34326.0), (5.0, 34323.0), (5.0, 33138.0), (5.0, 32840.0), (5.0, 32515.0), (5.0, 27792.0), (5.0, 26797.0), (5.0, 26749.0), (5.0, 26587.0), (5.0, 26501.0), (5.0, 26326.0), (5.0, 26131.0), (5.0, 8938.0), (5.0, 8675.0), (5.0, 8341.0), (5.0, 8261.0), (5.0, 8240.0), (5.0, 8123.0), (5.0, 7771.0), (5.0, 7502.0), (5.0, 7116.0), (5.0, 7096.0), (5.0, 7087.0), (5.0, 7024.0), (5.0, 6725.0), (5.0, 6219.0), (5.0, 6184.0), (5.0, 5866.0), (5.0, 5732.0), (5.0, 5475.0), (5.0, 5473.0), (5.0, 5071.0), (5.0, 5017.0), (5.0, 4969.0), (5.0, 4967.0), (5.0, 4822.0), (5.0, 4789.0), (5.0, 4783.0), (5.0, 4731.0), (5.0, 4711.0), (5.0, 4697.0), (

In [10]:
recommendedMovies=[]
for item in recommendedMovieIds:
    print(movies[item[1]])


7th Voyage of Sinbad, The (1958)
The Beatles: Eight Days a Week - The Touring Years (2016)
Making a Murderer (2015)
Helter Skelter (1976)
Willie & Phil (1980)
Forsyte Saga, The (1967)
Death on the Staircase (Soupçons) (2004)
Bleak House (2005)
Eaux d'artifice (1953)
Black Mirror (2011)
Mildred Pierce (2011)
Howl (2010)
Aria (1987)
Peter Pan (1960)
Not Quite Hollywood: The Wild, Untold Story of Ozploitation! (2008)
Manson Family, The (2003)
Z Channel: A Magnificent Obsession (2004)
Napoléon (1927)
My Winnipeg (2007)
O Lucky Man! (1973)
Cul-de-sac (1966)
Wild Zero (2000)
Last Days (2005)
Devil's Rejects, The (2005)
Palindromes (2004)
Vincent (1982)
Walker (1987)
Saddest Music in the World, The (2003)
Visions of Light: The Art of Cinematography (1992)
Prospero's Books (1991)
Decalogue, The (Dekalog) (1989)
Choose Me (1984)
Holy Mountain, The (Montaña sagrada, La) (1973)
Battle of Algiers, The (La battaglia di Algeri) (1966)
Tarnation (2003)
Enemy Below, The (1957)
Oliver Twist (1948)
3 Wo

Hidalgo (2004)
Before the Devil Knows You're Dead (2007)
Palm Beach Story, The (1942)
Crooklyn (1994)
Way We Were, The (1973)
Raise the Red Lantern (Da hong deng long gao gao gua) (1991)
Empire of the Sun (1987)
Muppet Movie, The (1979)
Snatch (2000)
Good Morning, Vietnam (1987)
Team America: World Police (2004)
Iron Monkey (Siu nin Wong Fei-hung ji: Tit Ma Lau) (1993)
Moon (2009)
Miller's Crossing (1990)
Ring, The (2002)
Man from Snowy River, The (1982)
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)
Deer Hunter, The (1978)
Life as a House (2001)
This Film Is Not Yet Rated (2006)
Go West (1925)
Perfect Candidate, A (1996)
Pirates of the Caribbean: The Curse of the Black Pearl (2003)
Beauty and the Beast (1991)
Lion King, The (1994)
Sid and Nancy (1986)
Fog of War: Eleven Lessons from the Life of Robert S. McNamara, The (2003)
Judgment at Nuremberg (1961)
Peyton Place (1957)
Saving Private Ryan (1998)
Big Sleep, The (1946)
Invasion of the Body Snatchers (1956)
Sweet and Lowdown (1999)
Cr

Pretty Baby (1978)
Brothers Bloom, The (2008)
Pinocchio (1940)
Clear and Present Danger (1994)
Beverly Hills Cop (1984)
Frida (2002)
My Fellow Americans (1996)
Confessions of a Dangerous Mind (2002)
Odd Couple, The (1968)
Awakenings (1990)
Ghost and the Darkness, The (1996)
Four Weddings and a Funeral (1994)
Lion in Winter, The (1968)
Battlestar Galactica: Razor (2007)
How Green Was My Valley (1941)
Bad Lieutenant (1992)
Madame Butterfly (1995)
Patch Adams (1998)
Bush's Brain (2004)
Charly (1968)
Avalon (1990)
V for Vendetta (2006)
Poltergeist (1982)
Snow White and the Seven Dwarfs (1937)
X-Files: Fight the Future, The (1998)
Night on Earth (1991)
Misery (1990)
Hotel Rwanda (2004)
Oklahoma! (1955)
Abyss, The (1989)
Star Trek VI: The Undiscovered Country (1991)
Wife, The (1995)
Bachelor Party (1984)
Dirty Pretty Things (2002)
Trainspotting (1996)
Tin Men (1987)
Sneakers (1992)
Jurassic Park (1993)
Opposite of Sex, The (1998)
How to Marry a Millionaire (1953)
From Hell (2001)
Greatest Sh

Bad Boys II (2003)
I.Q. (1994)
Problem Child (1990)
Senseless (1998)
Striking Distance (1993)
White Sands (1992)
Stanley & Iris (1990)
Horton Hears a Who! (2008)
Saw II (2005)
Cloud Atlas (2012)
Bride of Chucky (Child's Play 4) (1998)
Ace Ventura: When Nature Calls (1995)
Emerald Forest, The (1985)
Nuts (1987)
Hercules (1997)
Happy, Texas (1999)
Underworld (2003)
Grease 2 (1982)
Snakes on a Plane (2006)
Made in America (1993)
What Lies Beneath (2000)
Man Without a Face, The (1993)
EDtv (1999)
Kung Pow: Enter the Fist (2002)
Austin Powers in Goldmember (2002)
Mercury Rising (1998)
Mask, The (1994)
Life Aquatic with Steve Zissou, The (2004)
Spartan (2004)
Party Girl (1995)
Scary Movie (2000)
Rock Star (2001)
Young Doctors in Love (1982)
Kickboxer (1989)
Under the Skin (2013)
One Fine Day (1996)
Constantine (2005)
Prince of Tides, The (1991)
Meet Joe Black (1998)
Earth Girls Are Easy (1988)
Ace Ventura: Pet Detective (1994)
Surviving the Game (1994)
Drop Dead Fred (1991)
Grown Ups (2010)


AVPR: Aliens vs. Predator - Requiem (2007)
Butterfly on a Wheel (Shattered) (2007)
Wrong Turn 2: Dead End (2007)
Margot at the Wedding (2007)
Alvin and the Chipmunks (2007)
Golden Compass, The (2007)
Awake (2007)
Hitman (2007)
Mist, The (2007)
Darfur Now (2007)
Mr. Magorium's Wonder Emporium (2007)
Beowulf (2007)
Diving Bell and the Butterfly, The (Scaphandre et le papillon, Le) (2007)
Bee Movie (2007)
Martian Child (2007)
Elite Squad (Tropa de Elite) (2007)
My Kid Could Paint That (2007)
City of Violence, The (Jjakpae) (2006)
Scum (1979)
Saw IV (2007)
Tyler Perry's Why Did I Get Married? (2007)
Edge of Heaven, The (Auf der anderen Seite) (2007)
Silk (2007)
Control (2007)
Irina Palm (2007)
Assassination of Jesse James by the Coward Robert Ford, The (2007)
Weirdsville (2007)
Elizabeth: The Golden Age (2007)
We Own the Night (2007)
Heartbreak Kid, The (2007)
Seeker: The Dark Is Rising, The (2007)
Game Plan, The (2007)
Good Luck Chuck (2007)
Tekkonkinkreet (Tekkon kinkurîto) (2006)
I Coul

Masculin Féminin (1966)
Three Musketeers, The (1973)
Seven-Per-Cent Solution, The (1976)
Zelary (2003)
Night and the City (1950)
Inside Deep Throat (2005)
Fata Morgana (1971)
Lessons of Darkness (Lektionen in Finsternis) (1992)
Bitter Tears of Petra von Kant, The (bitteren Tränen der Petra von Kant, Die) (1972)
Salem's Lot (1979)
Nobody Knows (Dare mo shiranai) (2004)
Wedding Date, The (2005)
Hide and Seek (2005)
Are We There Yet? (2005)
Assault on Precinct 13 (2005)
White Sound, The (Das weiße Rauschen) (2001)
Summer Storm (Sommersturm) (2004)
Hobson's Choice (1954)
Memories of Murder (Salinui chueok) (2003)
Coach Carter (2005)
Appleseed (Appurushîdo) (2004)
Abbott and Costello Meet the Invisible Man (1951)
Wizards (1977)
Sergeant York (1941)
Imaginary Heroes (2004)
Phantom of the Opera, The (1989)
Sweet Liberty (1986)
In the Realms of the Unreal (2004)
Fat Albert (2004)
Assassination of Richard Nixon, The (2004)
Beyond the Sea (2004)
3-Iron (Bin-jip) (2004)
Blood and Black Lace (Sei 